In [1]:
import mne 
import numpy as np
import pandas as pd

# parameters
n_samples_per_miniepoch = 60
n_samples_ini = 20
n_miniepochs_per_trial = 60


# load data
bids_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids'
subject = 'S01'
fn_movie = f'sub-{subject}_Dynamic_preprocessed-epo.fif'
epochs_movie = mne.read_epochs(f'{bids_dir}/derivatives/preprocessed/{fn_movie}')
epochs_movie=epochs_movie[epochs_movie.metadata['condition']!='catch']

# initialize
n_trials = epochs_movie._data.shape[0]
n_channels = epochs_movie._data.shape[1]
n_timepoints = epochs_movie._data.shape[2]


Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo-1.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
504 matching events found
No baseline correction applied
0 projection items activated


In [100]:
onset_first_miniepoch = np.where(epochs_movie.times==0)[0][0]+n_samples_ini

# setup for metadata
miniepochs = []
condition = epochs_movie.metadata['condition']
direction = [c.split('_')[1] for c in condition]
right_idx = [d=='Right' for d in direction]
left_idx = [d=='Left' for d in direction]
degree = [int(c.split('_')[0]) for c in condition] 
run = epochs_movie.metadata['run_nr']

for i in range(n_miniepochs_per_trial):
    # onset shifts by 20 samples, going over the entire movie
    # remember the first mini epoch is not at image onset but 20 samples later
    onset = onset_first_miniepoch+i*n_samples_per_miniepoch
    me = mne.EpochsArray(epochs_movie._data[:,:,onset:onset+n_samples_per_miniepoch],info=epochs_movie.info)

    # work out the wrap-around in circular space (can't have numbers >360 or <0)
    deg = np.zeros(len(degree))
    deg[right_idx] = np.array(degree)[right_idx]+3*i
    deg[left_idx] = np.array(degree)[left_idx]-3*i

    deg[deg>359] = deg[deg>359]-360
    deg[deg<0] = 359-np.abs(deg[deg<0])

    # make pandas metadata
    df = pd.DataFrame()
    df['condition']=condition
    df['direction']=direction
    df['degrees']=deg
    df['timepoints']=i
    df['trialnum']=np.arange(n_trials)
    df['run'] = run

    me.metadata = df
    miniepochs.append(me)

# concatenate all timepoints
mini_epochs = mne.concatenate_epochs(miniepochs)

# save them
file_name = f'{bids_dir}/derivatives/preprocessed/sub-{subject}_miniEpochs_preprocessed-epo.fif'
mini_epochs.save(file_name, overwrite=True)


Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching 

In [94]:
np.where(np.isnan(mini_epochs._data))

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [1]:
import mne
import pandas as pd
import numpy as np
import glob
import pylab
import pandas as pd
import pickle
import matplotlib.pyplot as plt

stim_duration = 3
run_length = 4
first_run_file = 1
last_run_file = 4
S = 'S01'
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
#proj_path = "/misc/data12/sjapee/Sebastian-OrientationImagery/Data/"
#raw_dir = proj_path + '20240611/' + S + '/'
#logfile_dir = raw_dir + 'sheets/extracted_dataMovie'
bids_dir = bids_top_dir + f'bids_dir/sub-{S}/ses-1/meg/'

In [ ]:
#*****************************#
### PARAMETERS ###
#*****************************#

l_freq                      = 0.1
h_freq                      = 100
notch                       = 60
notch_max                   = 240
pre_stim_time               = -.2
post_stim_time              = 3.2
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'


import glob

dsets = []
for i in range(1, 5, 1):
    ds = f'run-{i:02}'  # Format the run number with leading zeros
    pattern = f'{bids_dir}*Dynamic_{ds}_meg.ds'  # Adjust the pattern to match your file name structure
    matches = glob.glob(pattern)
    
    if matches:
        dsets.append(matches[0])
    else:
        print(f"No files found for pattern: {pattern}")

print(dsets)



In [3]:
#New way of defining onsets not based on standard deviation but based on finding photodiode after the trigger
def defineOnsets(raw, events):
    log_statements = []
    pd_dat = raw[raw.ch_names.index('UADC016-2104')][0][0]
    if np.mean(np.abs(pd_dat))>0.6: # this should be fine but may not be good if there are a looot of zeros
        # zero-centering & making everything positive
        pd_dat = np.abs(pd_dat-np.mean(pd_dat[0:100]))
        # scaling channel to 0-1
        pd_dat = (pd_dat-np.min(pd_dat)) / (np.max(pd_dat)-np.min(pd_dat))

        print(pd_dat)

        # using the event, we are now looking in the next 50ms of the photodiode channel (digital) when the signal goes above 0.5
        t_samples = int(0.05/(1/raw.info['sfreq']))
        threshold =  .5

        # when we found the events we calculate the trigger delay. 
        pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
        pd_onsets = []
        pd_offsets = []

        for event in events:
            # Find onset
            onset_index = event[0] + np.argmax(pd_dat[event[0]:event[0] + t_samples] > threshold)
            pd_onsets.append(onset_index)

            # Find offset
            pd_offset_index = onset_index + np.argmax(pd_dat[onset_index:] < threshold)
            pd_offsets.append(pd_offset_index)
        trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        # sometimes this channel records ON as UP and sometimes as DOWN. This if-loop makes sure we always find the onsets
        if np.mean(trigger_delay)==0:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
            trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        log_statements.append(('Trigger delay removed (μ ± σ): %0.1f ± %0.1f ms')
            % (np.mean(trigger_delay), np.std(trigger_delay)))
        if np.mean(trigger_delay)>10:
            events[:,0] = events[:,0]+5
            log_statements.append(f'photo diode and parallel port triggers are too far apart. using parallel port trigger ')
        else:
            log_statements.append(f'photo diode used successfully ')
            events[:, 0] = pd_events
    else: 
        # if the pd channel didn't work, add 5 samples as a standard delay
        events[:,0] = events[:,0]+5
        log_statements.append(f'photo diode did not record anything. using parallel port trigger ')

    return raw,events,log_statements, pd_onsets, pd_offsets

def notch_filter(raw):
    #apply notch filter based on global parameters set at the top
    notches = np.arange(notch, notch_max+1, notch)
    print(notches)
    raw.notch_filter(notches, phase='zero-double', fir_design='firwin2')
    return raw
def prepMarkers():
    subs   = ['S01']
    block_conditions = {
        'Watch_Still': ['0022_Left', '0022_Right', '0067_Left', '0067_Right', '0112_Left', '0112_Right', 
                        '0157_Left', '0157_Right', '0202_Left', '0202_Right', '0247_Left', '0247_Right',
                        '0292_Left', '0292_Right', '0337_Left', '0337_Right']}
    trial_data = []
    blocks = list(block_conditions.keys())
    for block in blocks:
        conditions = block_conditions[block]
        for condition in conditions:
            block_data = {'Block': block, 'Condition': condition}
            trial_data.append(block_data)

    df = pd.DataFrame(trial_data)
    print(df)
    df['Code'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    ids = df['Code']
    conds = df['Block']
    orients = df['Condition']
    event_id = {}
    for i, id in enumerate(ids):
        event_id[id] = f'{subs[0]}/{conds[i]}/{orients[i]}'
    #keys = [f'{sub}/{cond}/{orient}/{id}' for sub in subs for id in ids for cond in conds for orient in orients]
    return event_id

In [4]:
event_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [ ]:
print(event_id)

In [178]:
def transform_code(code):
    if code in {1, 2}:
        return 22
    elif code in {3, 4}:
        return 67
    elif code in {5, 6}:
        return 112
    elif code in {7, 8}:
        return 157
    elif code in {9, 10}:
        return 202
    elif code in {11, 12}:
          return 247
    elif code in {13, 14}:
          return 292
    elif code in {15, 16}:
          return 337

def add_additional_events(events):
    new_events_list = []
    for event in events:
        original_code = event[2]
        transformed_code = transform_code(original_code)
        new_events_list.append([event[0], 0, transformed_code])  # Add the original event with the transformed code
        
        onset = event[0]
        print('onset', onset)
        current_code = transformed_code
        increment = original_code % 2 == 0  # True if original_code is odd, False if even
        
        for i in range(1, 60):
            new_onset = onset + 20 + (i * 60)
            if increment:
                current_code = (current_code + 3) % 360
            else:
                current_code = (current_code - 3) % 360
                if current_code < 0:
                    current_code = 359
            new_events_list.append([new_onset, 0, current_code])

    print(new_events_list)
            
    return np.array(new_events_list)

In [ ]:
from mne.preprocessing import ICA


all_epochs = []
all_events = []
all_onsets = []
all_offsets = []
initial_events = []
for i in range(len(dsets)):
    raw = mne.io.read_raw_ctf(dsets[i],system_clock='ignore',preload=True)
    events = mne.find_events(raw, stim_channel='UPPT001',min_duration = 0.002)
    #print('events', events)
    #print('events2', events[-1])
    events = events[events[:, 2] != 17]
    raw, events, log_statements, onsets, offsets = defineOnsets(raw, events)
    #raw, annotations = writeMarkers(event_ids, raw, events, stim_duration, dsets[i])
    print('onset list', onsets)
    initial_events.append(events)
    events = add_additional_events(events)
    # remove trailing zeros
    endtime = raw.times[events[-1,0]]+10
    raw.crop(0,endtime)
    raw = notch_filter(raw)
    
    #ICA thing 
    epochs = mne.Epochs(raw, events, event_id=None,tmin=0, tmax=1/60, preload=True, baseline=None, picks = 'mag')
    print("Number of events:", len(events))
    print("Number of epochs:", len(epochs))
    print(np.nonzero(list(map(len, epochs.drop_log)))[0])
    [n for n, dl in enumerate(epochs.drop_log) if len(dl)]
    all_epochs.append(epochs)
    all_events.append(events)
    all_onsets.append(onsets)
    all_offsets.append(offsets)

print(len(all_events))

In [ ]:
print(all_events[3][50:70])

In [ ]:
print(epochs._data.shape)

In [ ]:
dev_head_t_ref = all_epochs[0].info['dev_head_t']

for i in range(0, 4):
    all_epochs[i].info['dev_head_t'] = dev_head_t_ref

epochs_stacked = mne.concatenate_epochs(all_epochs)

In [ ]:
for i in range(62):
    print('time', events[i, 0])
    print(events[i, 2])
print(epochs_stacked.events)

In [184]:
import numpy as np
import pandas as pd

events = epochs_stacked.events

# Initialize lists to store metadata
condition_list = []
frame_list = []
timepoint_list = []

# Iterate through events to create metadata
for i in range(len(events)):
    code = events[i, 2]  # Get the event code from the third column

    # Determine condition based on grouping every 60 entries
    if i % 60 == 0:

        condition = f'condition_{code}'  # Use the code of the first entry in the grouping
    else:
        condition = condition_list[-1]  # Use the last condition label if not a new condition
    
    # Frame is the event code itself
    frame = code
    
    
    # Append to lists
    condition_list.append(condition)
    frame_list.append(frame)

In [ ]:
print(all_events[0][50:70])

In [191]:
import numpy as np
import pandas as pd

events = epochs_stacked.events

# Initialize lists to store metadata
condition_list = []
frame_list = []
direction_list = []

# Iterate through events to create metadata
for i in range(len(events)):
    code = events[i, 2]  # Get the event code from the third column
    




    if i % 60 == 0:
        condition = f'condition_{code}'  # Use the code of the first entry in the grouping
    else:
        condition = condition_list[-1]  # Use the last condition label if not a new condition
    
    # Frame is the event code itself
    frame = code
    
    
    # Append to lists
    condition_list.append(condition)
    frame_list.append(frame)

# Create metadata dictionary
metadata = {
    'condition': condition_list,
    'frame': frame_list,
}

metadata_df = pd.DataFrame(metadata)


In [ ]:
epochs_stacked.metadata = metadata_df

In [ ]:
print(epochs_stacked.metadata[60:120])

In [ ]:
raw_mag = raw.pick_types('mag', ref_meg=False)

In [ ]:
print(initial_events)

In [ ]:
initial_events = np.array(initial_events)

# Data dimensions
num_events = initial_events[0].shape[0]*4
print(num_events)
num_sensors = raw_mag._data.shape[0]
# Calculate the indices for 240 samples before each event
indices_240_before = np.round(initial_events[:, 1] - 240).astype(int)

first_elements_master = []
for i in initial_events:

    first_dimension_list = [row.tolist() for row in i]
# Initialize an array to store the averaged data
    first_elements = [sublist[0] for sublist in first_dimension_list]
    first_elements_master.extend(first_elements)
events_minus_240 = [x - 240 for x in first_elements_master]
print(first_elements_master)
print(events_minus_240)

averaged_data = np.zeros((num_events, num_sensors))

# Loop through each index directly
for i, event_index in enumerate(events_minus_240):
    start_index = event_index
    end_index = event_index + 240
    if end_index > raw_mag._data.shape[1]:
        end_index = raw_mag._data.shape[1]
    
    # Extract the data and average it across the specified window
    data_window = raw_mag._data[:, start_index:end_index]
    averaged_data[i, :] = np.mean(data_window, axis=1)

# averaged_data now contains the averaged data for each event


print(averaged_data.shape)

In [198]:
expanded_array = np.repeat(averaged_data, 60, axis=0)

In [ ]:
expanded_array.shape

In [ ]:
print(epochs_stacked._data.shape)

In [ ]:
print(epochs_stacked._data[0])

In [202]:
epochs_final = epochs_stacked._data - expanded_array[:, :, np.newaxis]

In [203]:
epochs_stacked._data = epochs_final

In [ ]:
print(epochs_stacked._data[0])

In [205]:
file_name = 'miniEpochsnobaselineTest1-epo.fif'
epochs_stacked.save(file_name, overwrite=True)